# Training a DCGAN II
## (Deep Convolutional Generative Adversarial Networks)

This is adapted from [this TF tutorial](https://www.tensorflow.org/tutorials/generative/dcgan), as well as these: [the Chollet notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter12_part05_gans.ipynb), itself a port of [this Keras tutorial](https://keras.io/examples/generative/dcgan_overriding_train_step/), and [the paper](https://arxiv.org/pdf/1511.06434.pdf).

#### Install Imageio (to generate GIFs at the end)

```bash
conda install -c conda-forge imageio # locally (ships with Colab)
```

In [ ]:
import os
import sys
import PIL
import time
import glob
import imageio

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from IPython import display

## The [CelebA](https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) Dataset: Workflow

### 1. Colab

My recommendation is the following:

1. Download the dataset from the Google Drive of the authors **once**, and upload the `img_align_celeba.zip` file (1.4 GB!) to your drive.  
    1. Manually: [here](https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684)  
    2. Using `gdown`: `!gdown 1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684`

2. Each time you train, download to **the cloud machine**, unzip and load the images from there (rather than from an unzipped version of your drive, it's *way* faster). Be nice, and use your Google drive version instead of the authors. For that, you will need to go to the file in your Google Drive, click the `⋮` on the right, share, General access, "Anyone with the link". That link contains the id that you can use in in this code (or the equivalent code cell below):

```bash
!mkdir -p datasets/dcgan_celeba
!gdown <ID-OF-YOUR-CELEBA-COPY> -O datasets/dcgan_celeba/img_align_celeba.zip
!unzip -qq datasets/dcgan_celeba/img_align_celeba.zip -d datasets/dcgan_celeba
```

Perform these steps first, *then* connect to your drive and switch directories (if you want to save your model and generated images in your drive, otherwise no need).

### 2. Locally

Perform the steps to download the data once and unzip it so your directory looks like `DMLAP/python/datasets/dcgan_celeba/img_align_celeba` (using the lines above or the cell below).

To install [gdown](https://pypi.org/project/gdown/): `conda install -c conda-forge gdown`.

In [ ]:
import gdown
from zipfile import ZipFile

celeba_dir = "datasets/dcgan_celeba"
extracted_dir = os.path.join(celeba_dir, "img_align_celeba")

le_id = None # add your ID here

if not os.path.isdir(extracted_dir): 
    if le_id is None:
        print("Variable `le_id` is None: upload the Celeba dataset to your drive, retrieve its id, and add it to `le_id`!")
    else: 
        print("Downloading Celeba dataset")
        os.makedirs(celeba_dir, exist_ok=True)

        fname = "datasets/dcgan_celeba/data.zip"
        url = f"https://drive.google.com/uc?id={le_id}"
        gdown.download(url, fname, quiet=False)

        print("Unzipping")
        with ZipFile("datasets/dcgan_celeba/data.zip", "r") as zipobj:
            zipobj.extractall("datasets/dcgan_celeba")
else:
    print("CelebA directory exists")

Then you can import your files like so:

In [ ]:
BATCH_SIZE = 32    # you can push this up if you have a good GPU
IMAGE_SIZE = 64    # reducing this could help speed up training, but the convnet
                   # architecture would have to be adapted (in generator/discriminator)
IMAGE_CHANNELS = 3 # 1 Grayscale (faster), 3 RGB
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

LATENT_DIM = 100 # The size of the latent space/input vector

In [ ]:
# utils 

def norm(x):
    """Normalize the inputs to [-1, 1] (generator with 'tanh' activation)"""
    return (x - 127.5) / 127.5

def denorm(x):
    """Denormalize the outputs from [-1, 1] to [0,255] (generator with 'tanh' activation)"""
    return (x + 1) * 127.5

See [Load and preprocess images](https://www.tensorflow.org/tutorials/load_data/images).

In [ ]:
image_ds_dir = os.path.join(celeba_dir, "img_align_celeba")

# this often fails on Colab (timeout reading file), just rerun the cell
train_dataset = tf.keras.utils.image_dataset_from_directory(
    image_ds_dir,
    label_mode=None,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    smart_resize=True,
    color_mode="rgb" if IMAGE_CHANNELS == 3 else "grayscale" 
)
train_dataset = train_dataset.map(lambda x: norm(x))  # normalize the images to [-1, 1]

In [ ]:
ds_len = len(train_dataset)
print(f"{ds_len * BATCH_SIZE} samples in {ds_len} batches")

Now let&rsquo;s see one random instance from the dataset:

In [ ]:
for x in train_dataset:
    a = tf.cast(denorm(x[0]), tf.int32)
    # print(tf.reduce_min(a), tf.reduce_max(a))
    plt.imshow(a, cmap="gray" if IMAGE_CHANNELS == 1 else None)
    break

#### Note

Now you could mount your drive and switch directory if you wanted to.

```python
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/')  # 'My Drive' is the default name of Google Drives
    os.chdir('drive/My Drive/2023-DMLAP/DMLAP/python') # change to your favourite dir
```

## The generator


At this stage we need to construct our generator. There are many implementations out there, this is one that works reasonably well for our use case and it is adapted from [here](https://github.com/Kaustubh1Verma/Art-using-GANs/blob/ff41eeb5099d2aa3976ed1f051596d14015548d5/DCGAN/DCGAN.py).
For GAN models it is [recommended](https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/) to initialize the layers with normally distributed (Gaussian) values and standard deviation of 0.02. The following function is built to enable training on different image sizes, but we recommend sticking with the `64x64` image size. You can vary the value of `kernel_size` to either `3`, `4` and `5` and see how that affects your image quality. A larger kernel size might produce better images in some cases, but will result in slower training. The code also has comments where you can experiment with modifications

In [ ]:
kernel_size = 3  # Set to either 3, 4 or 5

# This will enable working with image sizes other than 64x64 (untested)
upsample_layers = 5 # Needs to be the number of gen_block below
start_size = IMAGE_SIZE // (2**upsample_layers)
starting_filters = 128

# This will initialize our layers randomly
init = lambda: tf.keras.initializers.RandomNormal(stddev=0.02)

# Define a generator convolutional block
def gen_block(size, batch_norm=True):
    return tf.keras.Sequential(
        [   # UpSampling2D + Conv2D instead of Conv2DTranspose
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(size, kernel_size, padding="same", kernel_initializer=init(), use_bias=False),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.BatchNormalization(momentum=0.9),
        ]
    )

generator = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(LATENT_DIM,)),
        tf.keras.layers.Dense(starting_filters * start_size * start_size, activation="relu"),
        tf.keras.layers.Reshape((start_size, start_size, starting_filters)),
        tf.keras.layers.BatchNormalization(momentum=0.9),
        gen_block(1024),
        gen_block(512),
        gen_block(128),
        gen_block(64),
        gen_block(32),
        tf.keras.layers.Conv2D(IMAGE_CHANNELS, kernel_size=kernel_size, padding="same",
                               activation="tanh", kernel_initializer=init()),
    ]
)

generator.summary(expand_nested=True, line_length=100)

Let&rsquo;s see it&rsquo;s output before training



In [ ]:
latent_vector = tf.random.normal([1, LATENT_DIM])
generated_image = generator(latent_vector, training=True)
generated_image = tf.cast(denorm(generated_image), tf.int32)
print(generated_image.shape, generated_image.dtype, tf.reduce_min(generated_image), tf.reduce_max(generated_image))
plt.imshow(generated_image[0], cmap="gray" if IMAGE_CHANNELS == 1 else None)
plt.show()

## The discriminator

We then define the discriminator model, again adapted adapted from [here](https://github.com/Kaustubh1Verma/Art-using-GANs/blob/ff41eeb5099d2aa3976ed1f051596d14015548d5/DCGAN/DCGAN.py).



In [ ]:
# Define a discriminator convolutional block
def disc_block(size, strides, batch_norm=True, padding=False, dropout=False):
    layer_list = [
        tf.keras.layers.Conv2D(
            size, kernel_size=kernel_size, strides=strides, padding="same",
            kernel_initializer=init(), use_bias=not batch_norm
        ),  # Always keep
        tf.keras.layers.LeakyReLU(alpha=0.2),  # Always keep
    ]
    if dropout:
        layer_list += [ # Try varying the Dropout probabilty, keep the value <= 0.5
            tf.keras.layers.Dropout(0.3)
        ] 
    if padding:
        layer_list += [ # Always keep this
            tf.keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1)))
        ]
    if batch_norm:
        layer_list += [tf.keras.layers.BatchNormalization(momentum=0.8)]
    return tf.keras.Sequential(layer_list)


discriminator = tf.keras.Sequential(
    [
        tf.keras.Input(shape=IMAGE_SHAPE),
        tf.keras.layers.GaussianNoise(0.1),  # Injects noise into discriminator. Try varying noise amount <= 0.2, or removing by commenting this line
        disc_block(32, 1, batch_norm=False),  # Try varying the dropout flag in these layers
        disc_block(64, 2, batch_norm=True, padding=True),
        disc_block(128, 2, batch_norm=True),
        disc_block(256, 2, batch_norm=True),
        disc_block(512, 2, batch_norm=False, dropout=True),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

discriminator.summary(expand_nested=True)

In [ ]:
# Experiment with changing this, lower values result in slower but more stable learning 
# You could also have two different losses for G/D, if you want to control the learning separately
learning_rate = 0.0001

# on TF metal you might want to switch to tf.keras.optimizers.legacy.Adam
generator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5, beta_2=0.9) 
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5, beta_2=0.9)

### Save checkpoints

This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted. See [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) as well as [Training checkpoints](https://www.tensorflow.org/guide/checkpoint).

In [ ]:
model_dir = 'models/dcgan_celeba'
 
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

generated_dir = os.path.join(model_dir , "generated")

if not os.path.isdir(generated_dir):
    os.mkdir(generated_dir)

In [ ]:
checkpoint = tf.train.Checkpoint(
    epoch=tf.Variable(0), # we save the epoch to be able to resume training
    generator=generator,
    generator_optimizer=generator_optimizer,
    discriminator=discriminator,
    discriminator_optimizer=discriminator_optimizer,
)

manager = tf.train.CheckpointManager(checkpoint, os.path.join(model_dir, "ckpt"), max_to_keep=3)

# This will automatically restore the latest checkpoint: you must delete the ckpt files for this not to happen
checkpoint.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print(f"Restored from {manager.latest_checkpoint}")
else:
    print("Initializing from scratch.")

## Train the network

In [ ]:
# utils

def generate_images(generator, batch, latent_vectors=None, save=True):
    if latent_vectors is None: # if no latent vector is passed, create one
        latent_vectors = tf.random.normal(shape=(1, LATENT_DIM))
    # Notice `training`= False, so that the model runs in inference mode (doesn't influcence training + batchnorm)
    generated_images = generator(latent_vectors, training=False)
    generated_images = tf.cast(denorm(generated_images), tf.int32)
    
    if save:
        for i, gen_img in enumerate(generated_images):
            img = tf.keras.preprocessing.image.array_to_img(gen_img)
            img.save(os.path.join(generated_dir, f"e{epoch+1:03}_{batch:04}_generated_img_{i+1}.png"))
    
    return generated_images


def plot(g_losses, d_losses, generator, generated_images=None, clear=False,
         cmap="gray" if IMAGE_CHANNELS == 1 else None, save=False):
    """
    Book-keeping:
    Visualize losses and one example image for the epoch
    """
    if clear:  # You may want to clear the screen for longer training
        display.clear_output(wait=True)
    plt.figure(figsize=(10,5))
    
    # losses
    plt.subplot(1,2,1)
    plt.title('Losses')
    plt.plot(g_losses, label='Generator')
    plt.plot(d_losses, label='Discriminator')
    plt.legend()
    
    if generated_images is None:
         generated_images = generate_images(generator, save=save)
    # image
    plt.subplot(1,2,2)
    plt.axis('off')
    plt.imshow(generated_images[0], cmap=cmap) # [0]: remove the batch dimension  
    plt.show()

### Losses & train step

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy() 

smoothing = 0.1   # Label smoothing (keep < 0.2), can improve results. Setting to zero will disable smoothing 

def discriminator_loss(real_output, fake_output):
    # real_loss will quantify our loss to distinguish the real images
    real_labels = tf.zeros_like(real_output) + smoothing * tf.random.uniform(tf.shape(real_output))
    real_loss = cross_entropy(real_labels, real_output)

    # fake_loss will quantify our loss to distinguish the fake images (generated)
    fake_labels = tf.ones_like(fake_output) - smoothing * tf.random.uniform(tf.shape(fake_output))    
    fake_loss = cross_entropy(fake_labels, fake_output)
    
    # Real image = 0, Fake image = 1 (array of ones and zeros)
    total_loss = real_loss + fake_loss
    return total_loss, real_loss, fake_loss


def generator_loss(fake_output):
    # We want the false images to be seen as real images (0s)
    return cross_entropy(tf.zeros_like(fake_output), fake_output)


@tf.function # optimize for faster execution
def train_step(images):
    batch_size = tf.shape(images)[0]
    latent_vector = tf.random.normal([batch_size, LATENT_DIM])

    # To make sure we know what is done, we will use a gradient tape instead of compiling
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Training the generator
        generated_images = generator(latent_vector , training=True) 

        # Training the discriminator
        real_output = discriminator(images, training=True)           # Training the discriminator on real images
        fake_output = discriminator(generated_images, training=True) # Training the discriminator on fake images

        # Calculating the losses
        gen_loss =  generator_loss(fake_output)
        disc_loss, disc_r_loss, disc_f_loss = discriminator_loss(real_output, fake_output)

        # Building the gradients
        gradients_of_generator =     gen_tape.gradient(  gen_loss,  generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        
        # Applying the gradients (backpropagation)
        generator_optimizer.apply_gradients(    zip(gradients_of_generator,     generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        return gen_loss, disc_loss, disc_r_loss, disc_f_loss

The loop will save images and models for each epoch to a directory specifed in the variable `model_dir`. You can visualize the results by examining the directory. Keep note of this directory, because you will use that to load and visualize images in the [06_visualizing_gan_results.ipynb](05_visualizing_gan_results.ipynb) notebook. You may want to name the directory in a way that reminds you of the parameters you used for training. As usual, examine the code below and look out for comments that indicate paramters that you can modify.

In [ ]:
EPOCHS = 60        # Number of epochs. Safe to increase but check what happens to the images

save_every = 10    # (Epoch) How often to save the model
gen_every = 50     # (Batch) How often we generate images (Colab is faster: 100?)
print_every = 20   # (Batch) How often we print the loss (Colab is faster: 50?)

max_batch = 300    # (Batch) skip to next epoch after limited number of batches, -1 to disable

# Training loop
g_losses = []
d_losses = []

# Creating the vector once at the start provides some stability to our images
# Leaving it as None will create different seeds at each iteration (can be good
# to see if mode collapse is happening or not)
init_latent_vectors = None # tf.random.normal([1, LATENT_DIM])

start_epoch = checkpoint.epoch.numpy()

n = train_dataset.cardinality().numpy().item() # The number of batches per epoch

for epoch in range(start_epoch, start_epoch + EPOCHS):
    
    # Iterate over all batches
    batch_d_losses = []
    batch_g_losses = []
    for i, batch in enumerate(train_dataset): 
        
        # limit computation
        if i == max_batch:
            break
        
        # Upadte parameters for this batch
        g_loss, d_loss, r_loss, f_loss = train_step(batch)
        
        # Store losses for batch, we will average these for the whole epoch for a more stable visualization
        batch_g_losses.append(g_loss)
        batch_d_losses.append(d_loss)
        
        # Some printing
        if i % print_every == 0:
            dl, gl, rl, fl  = d_loss.numpy(), g_loss.numpy(), r_loss.numpy(), f_loss.numpy()
            print(f"epoch {epoch+1}, batch {i:{len(str(n))}/{n} [D loss: {dl:.4f} (real: {rl:.4f}, fake: {fl:.4f}) | G loss: {gl:.4f}]")
        
        # Generating/Saving images
        if i % gen_every == 0:
            generated_images = generate_images(generator, i, latent_vectors=init_latent_vectors)
             
    print()
    
    g_losses.append(np.mean(batch_g_losses))
    d_losses.append(np.mean(batch_d_losses))
    
    # Plot & save images
    plot(g_losses, d_losses, generator, generated_images=generated_images)

    # Saving model file (note the duplicate: this simplifies the loading in other files as we only use the generator
    if epoch > 0 and epoch % save_every == 0:
        print(f"{epoch+1}, saving model to {model_dir}")
        manager.save()
        generator.save(os.path.join(model_dir, f"e{epoch+1:03}_generator_celeba.keras")) # can be .h5

    checkpoint.epoch.assign_add(1) # increment our epoch        

## Create a GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no, img_no, resize_factor=6):
    s = IMAGE_SIZE * resize_factor
    return PIL.Image.open(
        os.path.join(generated_dir, f"e{epoch_no:03}_0000_generated_img_{img_no}.png")
    ).resize((s,s))

In [ ]:
display_image(checkpoint.epoch.numpy(), 1)

Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
import imageio as iio
import glob

anim_file = os.path.join(generated_dir, 'dcgan_celeba.gif')

if os.path.isfile(anim_file):
    print(f"An existing {anim_file}, found, removing")
    os.remove(anim_file)

n_image = 1       # I create only one series, but you could create several images / epoch
resize_factor = 6 # Make our image bigger
new_size = IMAGE_SIZE * resize_factor

# adapting the the tutorial version to v3 + looping the gif (thanks ChatGPT)
with iio.get_writer(anim_file, mode='I', loop=0) as writer:
    filenames = glob.glob(os.path.join(generated_dir, f"e*_generated_img_{n_image}.png"))
    filenames = sorted(filenames)
    for filename in filenames:
        # Use PIL to open and resize the image
        with PIL.Image.open(filename) as img:
            img_resized = img.resize((new_size, new_size))
            writer.append_data(np.array(img_resized))      

In [ ]:
# adapted from here: https://github.com/tensorflow/docs/blob/master/tools/tensorflow_docs/vis/embed.py

import base64
import pathlib
import mimetypes
import IPython.display

def embed_data(mime, data):
    """Embeds data as an html tag with a data-url."""
    b64 = base64.b64encode(data).decode()
    if mime.startswith('image'):
        tag = f'<img src="data:{mime};base64,{b64}"/>'
    elif mime.startswith('video'):
        tag = textwrap.dedent(f"""
            <video width="640" height="480" controls>
              <source src="data:{mime};base64,{b64}" type="video/mp4">
              Your browser does not support the video tag.
            </video>
            """)
    else:
        raise ValueError('Images and Video only.')
    return IPython.display.HTML(tag)

def embed_file(path):
    """Embeds a file in the notebook as an html tag with a data-url."""
    path = pathlib.Path(path)
    mime, unused_encoding = mimetypes.guess_type(str(path))
    data = path.read_bytes()
    return embed_data(mime, data)

embed_file(anim_file)

---

## Experiments

The work that can be done here broadly falls into three main directions:
- *Freeze* the network, work on the dataset:
  - In this direction, most of your work is to gather datasets, and improve the ease of use. Are you able to develop a suite of tools that would allow you to handle datasets more easily? (In this case, the images are already cropped and the same size, which already takes some work! It would be nice to integrate tools that allow you to make this part of the work more streamlined: put any images in a folder, and a Python script crops them, etc.)? It might be worth looking into [data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation) (inject randomness into your image dataset, [this tutorial](https://www.tensorflow.org/tutorials/generative/pix2pix) uses that).
  - It would be interesting to train GANs on generative images! You might end up with really distorted versions of what you started with.
  - It's likely that people have trained GANs on spectrograms, as we see now with diffusion, but it might be a real fun thing to try?
  - The image used for the week on text is a [book project by Allisson Parrish](https://www.aleator.press/releases/wendit-tnce-inf) that uses GANs to generate images of (unreadable) poems!
  - Also, people have created loops where they train GANs on their own outputs, which creates distortions that may be worth exploring.
- *Freeze* the dataset, work on the network:
  - Maybe there's one dataset that's really your focus, or you're happy to work with established material, or the whole data processing feels boring? You might then want to look into fiddling with the model, and gather tricks (for instance: do you see an improvement if you normalise your images to be between [0,1] instead of [-1,1], like here (your Generator will have to have a `sigmoid` rather than a `tanh` as its last layer)? Then of course there's the network themselves, where all sorts of parameters can be tweaked, from the number of layers, to the strides of the convolution...
  - **Note:** experimenting at a technical level with GANs (like with other things) can be a confusing rabbit hole. My recommendations are: make sure you have stable resources (e.g. you own a GPU or pay for Colab Pro), and try and make your net/dataset/experiments *as small/easy as possible*, so you can make a lot of them, get an inuition of what works and what doesn't. Perfect results really aren't the goal here, and it's never good for your momentum to have to wait hours or days before training finishes!
  - How do you document this process of experimentation? You would probably need to save the various parameters of your experimentation (for yourself and, perhaps, the viewer), and associate that with some images generated at this point.
- *Freeze* both network and dataset, and try to use the network, or its output, in unexpected ways: one could imagine just training this network, or using a top-level StyleGAN (see below), and using the resulting images in some way, as material for something else? 


## The State of the Art

The field has now moved away from GANs, as Diffusion has gained in popularity. The best results have probably been achieved by [Nvidia's StyleGan 3](https://nvlabs.github.io/stylegan3/) ([repo](https://github.com/NVlabs/stylegan3)) (both written in PyTorch). Check the [StyleGAN 3 notebook](10_models_1_stylegan3.ipynb) to check it out (on Colab!).

Another interesting option to look into is lucidrains' [Lightweight GAN](https://github.com/lucidrains/lightweight-gan) implementation.

## Zoos: list of all GAN variants

When it comes to GANs, just like Diffusion now, the explosion has been so enormous it is rather difficult (impossible?) to keep up:

- [Avinash Hindupur, "The GAN Zoo"](https://github.com/hindupuravinash/the-gan-zoo)
- [Jihye Back, "GAN-Zoos"](https://happy-jihye.github.io/gan/)

---

## Notes / Tricks

More resources worth checking: [Soumith Chintala, "How to Train a GAN? Tips and tricks to make GANs work"](https://github.com/soumith/ganhacks) (and [video](https://www.youtube.com/watch?v=X1mUN6dD8uE), as well as [Goodfellow's workshop](https://www.youtube.com/watch?v=HGYYEUSm-0Q)). This is summarised [in this part of a long course](https://www.youtube.com/watch?v=_cUdjPdbldQ&list=PLTKMiZHVd_2KJtIXOW0zFhFfBaJJilH51&index=153). To go deeper still, there's [this paper](https://arxiv.org/abs/1606.03498), and a [GAN guide](https://github.com/garridoq/gan-guide), and the [Art using GANs](https://github.com/Kaustubh1Verma/Art-using-GANs) repo.

Here is a summary of some of the tricks Chollet mentions in his book, that are used in this implementation:

- Sample from the latent space using a **normal distribution** (Gaussian), not a uniform one;
- GANs are likely to get stuck in all sorts of ways (it's an unstable, dynamic equilibrium): we introduce **random noise** to the labels for the discriminator to prevent this (called label smoothing);
- Sparse gradients can hinder GAN training, remedy: **strided convolutions** for downsampling instead of max pooling, and the **`LeakyReLU`** instead of `ReLu`;
- To avoid checkerboard artifacts caused by unequal coverage of the pixel space in the generator, use a kernel size **divisible by the stride size** with strided `Conv2DTranspose` or `Conv2D`. [In this implementation, we avoid those and use `UpSamling2D` followed by a `Conv2D`].

<small>*Deep Learning With Python*, 2<sup>nd</sup> ed., p.404</small>

Note also that, as is mentioned by Chintala (see lecture above), the labels for true/fake are reversed from the original formulation (here 0 is true, 1 is fake), that is said to improve stability.